In [1]:
from netCDF4 import Dataset
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import pandas as pd
import metpy.calc as mpcalc
from metpy.cbook import get_test_data
from metpy.plots import Hodograph, SkewT
from metpy.units import units
my_example_nc_file = 'EUREC4A_HALO_BAHAMAS-Radiometer_20200122a.nc'
fh = Dataset(my_example_nc_file, mode='r')
df_fdir = pd.read_csv('BBR_DirectFraction_Flight_20200122a_R0.dat',skiprows = 34,delim_whitespace=True)
df_fdw = pd.read_csv('BBR_Fdn_clear_sky_Flight_20200122a_R0.dat',skiprows = 34, delim_whitespace=True)
nas = pd.read_csv('EUREC4A_HALO_BAHAMAS-QL_20200122a.nas',skiprows=152, header=None,delim_whitespace=True)
nas.rename(columns=nas.iloc[0]).drop(nas.index[0],inplace=True)
fdw_time = df_fdw.loc[:,['sod','F_dw']]
fdir_time = df_fdir.loc[:,'f_dir']
db = pd.concat([fdw_time,fdir_time],axis=1)
db.set_index('sod',inplace=True)
time = fh.variables['TIME'][:] # time
time_bac = pd.DataFrame(time.T, columns = ['time'])
time_bac = time_bac/1000
time_bac

#print(fh.variables.keys())

C:\Users\spark.DESKTOP-D1NJRMK\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,time
0,53856.0
1,53856.1
2,53856.2
3,53856.3
4,53856.4
...,...
331755,87031.5
331756,87031.6
331757,87031.7
331758,87031.8


In [ ]:
nas

In [ ]:
fdw_time

In [2]:
array = np.arange(51732,87041, 1) #41249 77030  333190 rows × 1 columns
arr = array.repeat(10)
df_arr = pd.DataFrame(arr, columns = ['time_arr'])
df_arr.set_index('time_arr',inplace=True)
data = df_arr.join(db)
data_int= data.interpolate(method='linear', limit_direction='forward', axis=0)
# data_int.drop(data_int.index[0:12089],inplace=True)
data_int.reset_index(inplace=True)
data_int.columns=['time','F_dw','f_dir']  #start:22810,end:355999
#data_int[data_int.time==53856.0]
#data_int[data_int.time==87030]
df = data_int.iloc[21240:353000,:]
df.reset_index(inplace=True,drop=True)
df.drop(['time'],axis=1)

,F_dw,f_dir
0,825.184584,0.860369
1,825.201364,0.860384
2,825.218143,0.860400
3,825.234923,0.860416
4,825.251702,0.860431
...,...,...
331755,92.938942,0.842318
331756,92.938942,0.842318
331757,92.938942,0.842318
331758,92.938942,0.842318


In [5]:
ri = fh.variables['IRS_PHI'][:] # roll angle "deg"
#ro = pd.DataFrame(ri.T)
pi = fh.variables['IRS_THE'][:] # pitch angle "deg"
#po = pd.DataFrame(pi.T)
yi = fh.variables['IRS_HDG'][:] # yaw angle "deg"
#y = pd.DataFrame(yi.T)
szai = fh.variables['SUN_ALT'][:] # solar altitude angle
#sza = pd.DataFrame(szai.T)
saai = fh.variables['SUN_AZI'][:] # solar azimuth angle
#saa = pd.DataFrame(saai.T)
fdw = df.loc[:,'F_dw']
fdir = df.loc[:,'f_dir']
SWd = fh.variables['FDSC'][:]
SWD = pd.DataFrame(SWd.T)
SWD.columns=['swu']
r_off = -0.00012623
p_off = 0.00834372
r = ri+r_off
p = pi+p_off

import math
fwd = []
for i in szai:
    for j in saai:
        for k in r:
            for l in p:
                for m in yi:
                    for n in fdw:
                        for o in fdir:
                            eq1 = math.sin((90-i)*(math.pi/180))
                            eq2 = math.cos((90-i)*(math.pi/180))*math.sin(k*(math.pi/180))*math.sin((j-m)*(math.pi/180))
                            eq3 = math.cos((90-i)*(math.pi/180))*math.sin(l*(math.pi/180))
                            eq4 = math.cos(k*(math.pi/180))*math.cos((j-m)*(math.pi/180))
                            eq5 = math.sin((90-i)*(math.pi/180))*math.cos(l*(math.pi/180))*math.cos(k*(math.pi/180))
                            #factor = eq1/((eq2-eq3*(eq4+eq5))
                            factor = eq1/(eq2+eq3*eq4+eq5)
                            Fdw=o*n*factor+(1-o)*n
                            fwd.append(Fdw)

fdw
                            #correction_weighted = Fdw_attitude_correction(fdw,r,p,yi,szai,saai,r_off,p_off,fdir)
# corrected_FDSC = SWD.swu + correction_weighted
# swd_correct = pd.DataFrame(corrected_FDSC)
# swd_correct.columns = ['swd_corr']
# swd_correct

KeyboardInterrupt: 

In [4]:
import math
def Fdw_attitude_correction(fdw,ri,pi,yi,szai,saai,r_off,p_off,fdir):
    r = ri+r_off
    p = pi+p_off
    eq1 = math.sin((90-szai)*(math.pi/180))
    eq2 = math.cos((90-szai)*(math.pi/180))*math.sin(r*(math.pi/180))*math.sin((saai-yi)*(math.pi/180))
    eq3 = math.cos((90-szai)*(math.pi/180))*math.sin(p*(math.pi/180))
    eq4 = math.cos(r*(math.pi/180))*math.cos((saai-yi)*(math.pi/180))
    eq5 = math.sin((90-szai)*(math.pi/180))*math.cos(p*(math.pi/180))*math.cos(r*(math.pi/180))
    factor = eq1/(eq2+(eq3*eq4)+eq5)
    Fdw=fdir*fdw*factor+(1-fdir)*fdw
    return Fdw
ri = fh.variables['IRS_PHI'][:] # roll angle "deg"
#ro = pd.DataFrame(ri.T)
pi = fh.variables['IRS_THE'][:] # pitch angle "deg"
#po = pd.DataFrame(pi.T)
yi = fh.variables['IRS_HDG'][:] # yaw angle "deg"
#y = pd.DataFrame(yi.T)
szai = fh.variables['SUN_ALT'][:] # solar altitude angle
#sza = pd.DataFrame(szai.T)
saai = fh.variables['SUN_AZI'][:] # solar azimuth angle
#saa = pd.DataFrame(saai.T)
fdw = df.loc[:,'F_dw']
fdir = df.loc[:,'f_dir']
SWd = fh.variables['FDSC'][:]
SWD = pd.DataFrame(SWd.T)
SWD.columns=['swu']
r_off = -0.00012623
p_off = 0.00834372


Fdw_attitude_correction(fdw,ri,pi,yi,szai,saai,r_off,p_off,fdir)



# eq2 = math.cos((90-szai)*(math.pi/180))*math.sin(r*(math.pi/180))*math.sin((saai-yi)*(math.pi/180))
# eq3 = math.cos((90-szai)*(math.pi/180))*math.sin(p*(math.pi/180))
# eq4 = math.cos(r*(math.pi/180))*math.cos((saai-yi)*(math.pi/180))
# eq5 = math.sin((90-szai)*(math.pi/180))*math.cos(p*(math.pi/180))*math.cos(r*(math.pi/180))
# #factor = eq1/((eq2-eq3*(eq4+eq5))
# factor = eq1/(eq2+eq3*eq4+eq5)
# Fdw=fdir*fdw*factor+(1-fdir)*fdw

TypeError: Only length-1 arrays can be converted to Python scalars

In [ ]:
eq

In [ ]:
SWu = fh.variables['FUS'][:]
LWu = fh.variables['FDL'][:]
LWd = fh.variables['FUL'][:]
SWU = pd.DataFrame(SWu.T)
LWU = pd.DataFrame(LWu.T)
LWD = pd.DataFrame(LWd.T)
df_big = pd.concat([time_bac,SWD,swd_correct,SWU,LWU,LWD],axis =1)
df_big.columns = ['Time', 'ShortWave_down','ShortWave_down_corr', 'ShortWave_up','LongWave_down', 'LongWave_up']
df_big['Time'] = df_big.loc[:,'Time'].round(1)
df_big

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
#df_new_24jan = pd.read_csv('df_new_24jan.csv')
plt.figure(figsize=(18,12),dpi=500)
plt.subplot(311)
plt.plot(df_big.Time,df_big.ShortWave_down, 'r', label = 'solar downward',linewidth=1)
plt.plot(df_big.Time,df_big.ShortWave_down_corr, 'c', label = 'solar downward corrected',linewidth=1)
plt.title('BACARDI Pyranometer and pyrgeometer Up and Down from the flight on 22-01-2020')
plt.ylabel('Irradiance ($W/M^2$)',fontsize=14)
plt.xticks([], [])
plt.yticks(fontsize=14)
plt.legend(fontsize=10)
plt.subplot(312)
plt.plot(df_big.Time,df_big.LongWave_down, 'b', label = 'terrestrial downward',linewidth=1)
plt.ylabel('Irradiance ($W/M^2$)',fontsize=14)
plt.xticks([], [])
plt.yticks(fontsize=14)
plt.legend(fontsize=10)
plt.subplot(313)
plt.plot(df_big.Time,df_big.ShortWave_up, 'g', label = 'solar upward',linewidth=1)
plt.plot(df_big.Time,df_big.LongWave_up, 'm', label = 'terrestrial upward',linewidth=1)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.ylabel('Irradiance ($W/M^2$)',fontsize=14)
plt.xlabel('Time of day (seconds)',fontsize=14)
plt.legend(fontsize=10)
plt.show()

In [ ]:
from datetime import datetime
def convertSeconds(seconds):
    h = seconds//(60*60)
    m = (seconds-h*60*60)//60
    s = seconds-(h*60*60)-(m*60)
    return [h, m, s]
time = []
for i in df.Time:
    t = convertSeconds(i)
    time.append(t)
Time = pd.DataFrame(time)
Time.round(1)
Time.columns=['hours','minutes','seconds']
h = Time.loc[:,'hours'].astype(str).str.split('.')
m = Time.loc[:,'minutes'].astype(str).str.split('.')
#s = Time.loc[:,'seconds'].astype(str).str.split('.')
hr = []
for j,k in h:
    hr.append(j)
hour = pd.DataFrame(hr)
mins = []
for y,z in m:
    mins.append(y)
minute = pd.DataFrame(mins)
# sec = []
# for p,q in s:
#     sec.append(p)
# second = pd.DataFrame(sec)
df_big = pd.concat([hour.iloc[:,0:1],minute.iloc[:,0:1],Time.loc[:,'seconds']], axis =1)
df_big['date'] = np.repeat(20200122, len(df_big))
df_big.columns=['hour','minute','second','date']
#df_big.to_csv('df_big.csv')
#df_big.loc[df_big.hour >= 24, 'date']
#d = df_big[df_big.hour ==24]['date']
#print (d)
#df_big.second = df_big.second.round(1)
#df_big['time'] = df_big.date.astype(str).str.cat(df_big[['hour','minute','second']].astype(str))
#df_big['time'].astype(str).astype(int)
#df_big["time"]= df_big["time"].astype(str) 
# #KT19_22_jan.to_csv('KT19_22_jan.csv')
# my_date = []
# for m in df_big.time:
#     date = datetime.strptime(m, "%Y%m%d%H%M%S.%f")
#     my_date.append(date)
# my_date = pd.DataFrame(my_date)


In [ ]:
df_p = pd.read_csv('df_big.csv',sep=';') 
df_p['hour'][df_p['hour'] > 23] = 0
df_p['time'] = df_p.date.astype(str).str.cat(df_p[['hour','minute','second']].astype(str))
my_date = []
for e in df_p['time']:
    date = datetime.strptime(e, "%Y%m%d%H%M%S.%f")
    my_date.append(date)
my_date = pd.DataFrame(my_date)

In [ ]:
from datetime import datetime
def convertSeconds(seconds):
    h = seconds//(60*60)
    m = (seconds-h*60*60)//60
    s = seconds-(h*60*60)-(m*60)
    return [h, m, s]
time = []
for i in df.Time:
    t = convertSeconds(i)
    time.append(t)
Time = pd.DataFrame(time)
print(Time)

In [ ]:
from datetime import datetime
def convertSeconds(seconds):
    h = seconds//(60*60)
    m = (seconds-h*60*60)//60
    s = seconds-(h*60*60)-(m*60)
    return [h, m, s]
time = []
for i in df.Time:
    t = convertSeconds(i)
    time.append(t)
Time = pd.DataFrame(time)

Time.columns=['hours','minutes','seconds']
Time['hours'][Time['hours'] > 23] = 0
h = Time.loc[:,'hours'].astype(str).str.split('.')
m = Time.loc[:,'minutes'].astype(str).str.split('.')
s = Time.loc[:,'seconds'].astype(str).str.split('.')
hr = []
for j,k in h:
    hr.append(j)
hour = pd.DataFrame(hr)
mins = []
for y,z in m:
    mins.append(y)
minute = pd.DataFrame(mins)
sec = []
for p,q in s:
    sec.append(p)
second = pd.DataFrame(sec)
df_big = pd.concat([hour.iloc[:,0:1],minute.iloc[:,0:1],Time.loc[:,'seconds']], axis =1)
#df_big['date'] = np.repeat(20200122, len(df_big))
df_big.columns=['hour','minute','second']
df_big.second = df_big.second.round(1)
# #df_big.to_csv('df_big.csv')
df_big['time'] = df_big.hour.astype(str).str.cat(df_big[['minute','second']].astype(str))
# # df_big.to_csv('df_big.csv')
# data = pd.read_csv('df_big.csv')
# #data.time = int
# date_time=[]
# for r in data.time:
#     w = int(r)   
#     date_time.append(w)
#date_time
#df_big.to_csv('df_big.csv', sep='.')
#df_big["time"]= df_big["time"].astype(int) 
#df_big.loc[:,'time']
# #KT19_22_jan.to_csv('KT19_22_jan.csv')
my_date = []
for e in df_big['time']:
    date = datetime.strptime(e, "%H%M%S.%f")
    my_date.append(date)
my_date = pd.DataFrame(my_date)

In [ ]:
my_date

In [ ]:
import datetime
start = datetime.datetime(2020, 1, 22, 14, 57, 36,0)
end = datetime.datetime(2021, 1, 23, 0, 10, 31,0)
delta = end - start
time_delta = []
for i in range(delta.seconds + 1):
    time_delta.append(start + datetime.timedelta(seconds=i))
df_time_delta = pd.DataFrame(time_delta)
df_time_delta
#df_big = pd.concat([df_time_delta, navigation.iloc[:,4:5]], axis=1)
#df_big.dropna(inplace =True)
#df_big.columns = ['Time','Altitude']

#df_big


In [ ]:
def main():
    #millis=input("Enter time in milliseconds ")
    millis = int(millis)
    seconds=(millis/1000)%60
    seconds = int(seconds)
    minutes=(millis/(1000*60))%60
    minutes = int(minutes)
    hours=(millis/(1000*60*60))%24
    return millis

    #print ("%d:%d:%d" % (hours, minutes, seconds))

main()

In [ ]:
from datetime import timedelta
d = timedelta(418235000)
print (d)